In [2]:
import math
from IPython.display import display, Math
import json
from pydantic import BaseModel, Field
from collections import Counter
from fractions import Fraction
from include.generators.factorize_smarter import FactorisedNumber, FactorizeSmarter
import logging

fs = FactorizeSmarter()


In [2]:



class Polynomial(BaseModel):
    variable: str = Field(default="x", description="Variable of the polynomial")
    coefficents: list[Fraction] = Field(default=[], description="Exponents of the polynomial, where the first is the lowest order term, i.e x^0 (1)")
    coefficients_neg_exp: list[Fraction] = Field(default=[], description="Negative exponents of the polynomial, where the first is the lowest order term, i.e x^-1")
    def frac_as_str(self, fraction: Fraction):
        if  fraction.is_integer():
            return f"{int(fraction)}"
        else:
            return "\\frac{%g}{%g}" % (fraction.numerator, fraction.denominator)
    def latex_dump(self):

        terms = []

        for exponent in range(0, len(self.coefficents)):

            if self.coefficents[exponent] == 0 & exponent != 0:
                continue # There is no need to print the term, as it is 0, as long as it is not both coefficient and exponent
            elif self.coefficents[exponent] == 1 & exponent != 0:
                coefficient = "" # If exponent is not 0, and the coefficient is 1, we will not print it
            elif self.coefficents[exponent] < 0:
                coefficient = f"- {self.frac_as_str(abs(self.coefficents[exponent]))}" # Prettier way of writing -1
            else:
                coefficient = f"+ {self.frac_as_str(self.coefficents[exponent])}"


            if exponent == 0:
                var_exponent = " " # x^0 is equal to 1, so we don't print it as 1 * y is y
            elif exponent == 1:
                var_exponent = f"{self.variable} "
            else:
                var_exponent = f"{self.variable}^{exponent} "

            term = f"{coefficient}{var_exponent}"
            print(term)
            terms.insert(0, term)
        response = "".join(terms)
        if response[0] == "+":
            return response[1:] # We don't want the first +'
        return "".join(terms)
        print(terms)

display(Math(Polynomial(coefficents=[-0.5,-2,3]).latex_dump()))


<IPython.core.display.Math object>

In [23]:


def _permutation_loop(factorlist: list, t1_length: int):
    logging.basicConfig(level=logging.INFO)
    last_factor = 1
    for i in range(0, len(factorlist)):
        _cur_list = factorlist.copy()
        term_1 = _cur_list.pop(i)
        logging.debug(f"t1_length {i}/{len(_cur_list)-1}: {term_1}")

        if term_1 == last_factor:
            logging.debug(f"Skipping {i} {term_1} as it is the same as last factor {last_factor}")
            continue
        else:
            last_factor = term_1

            yield (term_1, math.prod(_cur_list))

            if t1_length > 1:
                logging.debug(f"Recursing with {t1_length-1} factors left")
                for _child_pair in _permutation_loop(_cur_list, t1_length-1):
                    yield term_1* _child_pair[0], _child_pair[1]

def permutator(factorlist: list):
    logging.debug(f"Factor primes {factorlist}")
    last_term = 1
    permutations = 1
    # Lets first sort out 1, as that is always an option
    yield (1, math.prod(factorlist))
    for t1_length in range(1, (len(factorlist)//2)+1):
        logging.info(f"Starting permutation loop with {t1_length} / {len(factorlist)//2} factors left")
        yield from _permutation_loop(factorlist, t1_length)

def find_sum_product_pair(sum: int| Fraction | FactorisedNumber, product: int | Fraction| FactorisedNumber ) -> tuple[Fraction, Fraction]:
    logging.basicConfig(level=logging.INFO)
    # First lets make certain we have a factorised number
    if isinstance(sum, Fraction) | isinstance(product, int):
        product = fs.factoize(product)
    if isinstance(product, Fraction) | isinstance(sum, int):
        sum = fs.factoize(sum)


    logging.debug(f"My factors are")
    logging.debug(f"Sum: {sum.value} factors {sum.factors}")
    logging.debug(f"Product: {product.value} factors {product.factors}")


    # Identify if both have the same sign (positive or negative)
    if  product.value > 0 :
        logging.debug("As product is positive, the terms must be added and both terms must have same number")
        target =abs(sum.value)
        logging.debug(f"Target {target}")
        for terms in permutator(product.factors):
            term1 = terms[0]
            term2 = terms[1]
            if term1 + term2 == target:
                if sum.value < 0:
                    logging.debug("Sum is negative, so both sums must be negative")
                    term1 = term1 * -1
                    term2 = term2 * -1
                logging.debug(f"Found a pair {term1} + {term2} = {sum.value} and {term1} * {term2} = {product.value}")
                return Fraction(term1), Fraction(term2)
        return Fraction(0), Fraction(0)
    elif product.value < 0:
        logging.debug("Product is negative, so we will need to subtrack one term from the other, so we +term1 -term2")
        target = sum.value
        logging.debug(f"Target {target}")
        for terms in permutator(product.factors):
            term1 = terms[0]
            term2 = terms[1]
            logging.debug(f"Trying {term1} - {term2} to be {target}")
            if term1 - term2 == target:
                logging.debug(f"Found a pair {term1} - {term2} = {sum.value} and {term1} * {term2} = {product.value}")
                return Fraction(term1), Fraction(-term2)
        return Fraction(0), Fraction(0)
    if sum.value < 0 and product.value < 0:
        pass


    #print(json.dumps(product.model_dump(), indent=4))

resp = find_sum_product_pair(-8, -180)
print(resp)

(Fraction(10, 1), Fraction(-18, 1))


In [4]:




def all_factors(product: int| Fraction | FactorisedNumber):
    # First lets make certain we have a factorised number
    if isinstance(sum, Fraction) | isinstance(product, int):
        product = fs.factoize(product)

    values = product.factors

    logging.debug(f"And it thinks that complexity is {product.complex_number}")
    logging.debug(f"Processing Complex number: {product.value}, product factors: {product.factors}")
    if not product.complex_number:
        logging.debug("Product is a prime number, so it only has one factor (it self)")
        return 1, values[0]
    if len(values) == 2:
        logging.debug("Product only has two factors")
        return values[0], values[1]

    # As we only have to take just under half of all numbers
    print(len(values)//2)
    term1 = permutator(product.factors)





In [6]:
def _permutation_loop(factorlist: list, t1_length: int):
    last_factor = 1
    for i in range(0, len(factorlist)):
        _cur_list = factorlist.copy()
        term_1 = _cur_list.pop(i)
        logging.debug(f"t1_length {i}/{len(_cur_list)-1}: {term_1}")

        if term_1 == last_factor:
            logging.debug(f"Skipping {i} {term_1} as it is the same as last factor {last_factor}")
            continue
        else:
            last_factor = term_1

            yield (term_1, math.prod(_cur_list))

            if t1_length > 1:
                logging.debug(f"Recursing with {t1_length-1} factors left")
                for _child_pair in _permutation_loop(_cur_list, t1_length-1):
                    yield term_1* _child_pair[0], _child_pair[1]

def permutator(factorlist: list):
    logging.debug(f"Factor primes {factorlist}")
    last_term = 1
    permutations = 1
    # Lets first sort out 1, as that is always an option
    yield (1, math.prod(factorlist))
    for t1_length in range(1, (len(factorlist)//2)+1):
        logging.info(f"Starting permutation loop with {t1_length} / {len(factorlist)//2} factors left")
        yield from _permutation_loop(factorlist, t1_length)



logging.basicConfig(level=5)
listofpairs = []
for pair in permutator(fs.factoize(180).factors):
    listofpairs.append(pair)

for i in listofpairs:
    logging.info(i)

INFO:root:(15, 12)


In [33]:
mylist = [ 1, 2, 3, 4, 5]
myvar = mylist.pop(3)
print(mylist)
print(myvar)

[1, 2, 3, 5]
4


In [ ]:
def next_floor(x: int):
    return math.floor(x)